<a href="https://colab.research.google.com/github/jj-snuIS/meta_sql/blob/main/%5BSQL_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_3%EC%A3%BC%EC%B0%A8%5D_%ED%8C%90%EB%8B%A4%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%95%B8%EB%93%A4%EB%A7%81_%EB%B2%A0%EC%9D%B4%EC%A7%81_jj_julian_lee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 설정

## 기본 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np

## BigQuery 연동을 위한 기본 설정

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'crested-drive-372522' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## BigQuery 데이터 불러오기
- `loginUsersDf`: 일간 로그인 유저 데이터프레임
- `ordersDf`: 일간 주문 정보 데이터프레임
- `salesByShopDf`: 매장별 누적매출 데이터프레임
- `shopsDf`: 픽업 매장 정보 데이터프레임

In [ ]:
query_job = client.query("""
    select *
    from `crested-drive-372522.my_temp.daily_login_users`
""")

loginUsersDf = query_job.to_dataframe()
loginUsersDf['login_date'] = loginUsersDf['login_date'].astype('datetime64') #datetime 타입으로 변환(외부에서 불러온 날짜데이터의 경우 datetime이 아닌 다른 형식으로 들어오는 경우가 종종 있음)
#------------------------------------------------------------------------------------------------------------------------------------------------#
query_job = client.query("""
    select *
    from `crested-drive-372522.my_temp.daily_orders`
""")

ordersDf = query_job.to_dataframe()
ordersDf['order_date'] = ordersDf['order_date'].astype('datetime64') 

#------------------------------------------------------------------------------------------------------------------------------------------------#
query_job = client.query("""
    select pickup_shop_name, sum(payment_amount) as sales, count(distinct user_id) as pu
    from `crested-drive-372522.localparm_dw.orders`
    group by pickup_shop_name
""")

salesByShopDf = query_job.to_dataframe()

#------------------------------------------------------------------------------------------------------------------------------------------------#
query_job = client.query("""
    select *
    from `crested-drive-372522.localparm_dw.pickup_shop`
""")
shopsDf = query_job.to_dataframe()

# 데이터 살펴보기

## 일부만 출력 - head(), tail() ⭐️
- colab에서 출력가능한 데이터프레임은 최대 20,000행 입니다.
- colab에서 데이터프레임을 출력할 경우 `filter` 기능도 사용할 수 있습니다. 같이 살펴볼까요?

In [ ]:
ordersDf.head(4)

## 타입 확인 - type()

In [ ]:
print(type(ordersDf)) # <class 'pandas.core.frame.DataFrame'>

## 데이터의 분포를 확인 - value_counts() ⭐️

In [ ]:
ordersDf['pickup_shop_name'].value_counts()

# << Series를 DataFrame으로 만들어서 보면 보기/다루기 편하죠 아래 코드를 '.' 단위로 나눠서 실행해 볼까요? >>
# ordersDf[['pickup_shop_name', 'product_id']].value_counts().to_frame().reset_index().rename(columns={0:'order_count'}).head()

## 데이터의 타입, 갯수, 각 컬럼별 정보를 확인 - df.info()
- 참고: Series의 경우 info() attribute이 없기때문에 사용할 수 없습니다.

In [ ]:
ordersDf.info()

## 컬럼별 요약 통계 제공 - df.describe() ⭐️
- 공식문서: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html
- include파라미터의 default설정은 `number`(수치형)이며, `object`(문자형), `all`(모두) 옵션 선택 가능 
- 문자형 컬럼에 대한 통계량
  - `count`: 데이터 개수(중복 포함)
  - `unique`: 데이터 개수(중복 제외)
  - `top`: 가장 많은 빈도수를 갖는 값
  - `freq`: top에서 관측된 값의 빈도수
- Timestamp 타입의 컬럼에게는 first, last 값도 통계량으로 함께 제공

In [ ]:
# << 수치형 컬럼에 대한 통계량 확인 >>
ordersDf.describe() 

# << 문자열 컬럼에 대한 통계량 확인 >>
# ordersDf.describe(include='object')

# << 전체 컬럼에 대한 통계량 확인 >>
# ordersDf.describe(include='all')


## 데이터프레임의 컬럼별 타입 확인 - df.dtypes
- 참고 : Pandas는 문자열 자료형을 `object라는` 이름으로 인식하고, Python은 `string`이라는 이름으로 인식

In [ ]:
print(ordersDf.dtypes)

order_id                    object
order_date          datetime64[ns]
order_year_month            object
product_id                  object
user_id                     object
payment_amount               Int64
payment_status              object
pickup_shop_name            object
dtype: object


## 데이터프레임의 (행x열) 크기 확인 - df.shape ⭐️
- 함수가 아닌 DataFrame의 속성(Attributes)을 확인하는 경우는 ()를 붙이지 않습니다.
- loc[], iloc[] 처럼 속성 사용시 대괄호가 붙는 경우도 있으니 속성별 사용방법은 공식문서를 참고해주세요 💪
- [참고] 공식문서 : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [ ]:
print(ordersDf.shape) # (행 크기, 열 크기)

## 데이터프레임의 컬럼명 확인 - df.columns

In [ ]:
print(ordersDf.columns)

# 데이터 정렬하기

## 인덱스 정렬 - sort_indx()
- 인덱스를 기준으로 데이터를 정렬합니다.
- 기본 정렬은 오름차순이며, `ascending=False`를 통해 내림차순 정렬 가능.

In [ ]:
# << 샵별 상품별 주문건수 데이터프레임 생성 >>
shopAndProductCountDf = ordersDf[['pickup_shop_name', 'product_id']].value_counts().to_frame().reset_index().rename(columns={0:'order_count'}) 

# << index 값을 기준으로 정렬 >>
shopAndProductCountDf.sort_index(ascending=False).head()

## 값 정렬 - sort_values() ⭐️
- 값을 기준으로 정렬되며 기준값은 by로 설정합니다.
- by에는 2개 이상의 컬럼을 설정할 수 있습니다.
- 기본 정렬은 오름차순이며, `ascending=False`를 통해 내림차순 정렬 적용 가능.

In [ ]:
# << 단일컬럼 정렬 >>
shopAndProductCountDf.sort_values(by='order_count', ascending=False).head()

# << 다중컬럼 정렬 >>
# shopAndProductCountDf.sort_values(by=['pickup_shop_name', 'order_count'], ascending=False).head()

# << 다중컬럼 정렬 - 컬럼별 정렬순서 설정 >>
# shopAndProductCountDf.sort_values(by=['pickup_shop_name', 'order_count'], ascending=[True, False]).head() # 컬럼별 정렬 개별적용도 가능

# 데이터 뽑아내기 & 핸들링 하기

## 데이터프레임에서 일부 컬럼을 선택 ⭐️
- 데이터프레임에서 하나의 컬럼을 선택하면 `Series` 형태가 됩니다.
- 데이터프레임에서 두개 이상의 컬럼을 선택하면 `DataFrame` 형태가 됩니다.

In [ ]:
# << 데이터프레임에서 단일컬럼 뽑아내기 >>
orderIdDf = ordersDf['order_id'] # 하나의 열만 선택해서 orderIdDf에 할당하는 경우
orderIdDf.head() # Series도 head, tail 메소드를 가지고 있기 때문에. DataFrame과 동일하게 사용 가능

# << orderIdDf의 타입 확인 >>
# print(type(orderIdDf))

In [ ]:
# << 데이터프레임에서 다중컬럼 뽑아내기 >>
subOrdersDf = ordersDf[['order_id', 'order_date', 'user_id', 'payment_amount']] # 다수의 컬럼을 선택할때는 리스트[] 안에 컬럼을 감싼 상태로 전달해야 합니다.
subOrdersDf.head(3)

# << subOrdersDf의 타입 확인 >>
# print(subOrdersDf.dtypes)
# print(type(subOrdersDf)) 

## df.copy() ⭐️
- 데이터프레임을 복제할 때 사용됩니다.
- copy() 없이 복제한 데이터프레임을 수정할 경우, 원본 데이터프레임도 함께 수정되므로 원본데이터의 형태 유지가 중요한 경우라면 사본 생성시 copy()를 사용해주세요. 

In [ ]:
shopsDf

,pickup_shop_name,region_1depth_name,region_2depth_name,region_3depth_name,h_code,x,y
0,픽업존(다이소 난곡사거리점),서울,관악구,신림동,1162076500,37.481460,126.914207
1,픽업존(메이비카페),서울,구로구,구로동,1153054000,37.484929,126.900259
2,픽업존(서울드림신용협동조합),서울,구로구,신도림동,1153051000,37.509568,126.882036
3,픽업존(유승상가),서울,양천구,목동,1147055000,37.539895,126.881942
4,픽업존(GS25 목동3동점),서울,양천구,목동,1147053000,37.546389,126.864435
5,픽업존(참맛부대찌개아구찜),서울,영등포구,문래동3가,1156060500,37.516276,126.897390
6,픽업존(GS영등포당산점),서울,영등포구,당산동5가,1156056000,37.532379,126.896801
7,픽업존(더현대서울),서울,영등포구,여의도동,1156054000,37.525191,126.929113


In [ ]:
shopsDfCopy1 = shopsDf
shopsDfCopy2 = shopsDf.copy()
shopsDfCopy3 = shopsDf[['pickup_shop_name', 'h_code', 'x', 'y']]

# << 각 데이터프레임의 메모리 주소 확인 >>
id(shopsDf), id(shopsDfCopy1), id(shopsDfCopy2), id(shopsDfCopy3)

(140276469311280, 140276469311280, 140276452476192, 140276452474944)

## df.loc[] ⭐️
- `Access a group of rows and columns by label(s) or a boolean array`
- 인덱스를 기준으로 행 데이터를 추출하는 방법
- 협업이 많고 규모가 큰 프로젝트 일수록 iloc[] 보다는, `컬럼명을 명시적으로 작성해주는 loc[]를 사용하는 편.`

In [ ]:
ordersDf.head(10)

In [ ]:
# << indexing >>
# index = 8 & 하나의 컬럼 선택
ordersDf.loc[8, 'order_id']

'eb251887-798b-4777-aff3-35e5620a9465'

In [ ]:
# << indexing >> 
# index = 8 & 여러개의 컬럼 선택
ordersDf.loc[8, ['order_id', 'order_date', 'product_id', 'payment_amount']]

In [ ]:
# << slicing >>
# 이때, [시장(포함): 끝(포함)] 된다는 점에 유의!
ordersDf.loc[4:7, 'order_id':'user_id']

In [ ]:
# << fnacy indexing >>
ordersDf.loc[4:8, ['order_id', 'user_id']]
ordersDf.loc[8, 'order_id':'user_id']

In [ ]:
# << boolean indexing >>
condition = ordersDf['product_id'] == '7680cebc-f48a-40f0-a2e9-b9d8c0354555'
ordersDf[condition]

In [ ]:
# << boolean indexing - 다중 조건 >>
condition1 = ordersDf['product_id'] == '7680cebc-f48a-40f0-a2e9-b9d8c0354555'
condition2 = ordersDf['order_date'] == '2022-07-15'
ordersDf.loc[condition1 & condition2] # and <<- 31개 행
ordersDf.loc[condition1 | condition2] # or  <<- 296개 행

## df.iloc[]
- `Purely integer-location based indexing for selection by position.`
- df.loc[] 와 사용방법이 동일하지만, 인덱스가 아닌 행/열 번호를 기준으로 데이터를 추출
- [참고] 인덱스와 행번호의 차이 🔥 
  - 인덱스는 보통 0부터 시작하지만 중간에 데이터가 삭제되면 언제든 변할 수 있으며 숫자가 아니라 문자열도 사용할 수 있음
  - 하지만 행/열 번호는 데이터의 순서를 따라가기 때문에 정수를 통해서만 데이터 조회/추출이 가능

In [ ]:
# << indexing >>
ordersDf.iloc[3, 3]

In [ ]:
# << slicing >>
# df.loc와 다르게 [시작(포함):끝(제외)] 된다는 점에 유의!
ordersDf.iloc[4:7, 0:4]

# << 비교용 - loc slicing >>
# 이때, [시작(포함): 끝(포함)] 된다는 점에 유의!
# ordersDf.loc[4:7, 'order_id':'user_id']

In [ ]:
# << fancy indexing >>
ordersDf.iloc[[4, 5, 6], 0:4]

## Series.where
- `Series.where(시리즈에 대한 조건문, 조건이 거짓일때 값)` 형태로 사용하며 
- 조건이 참인 경우에 대해서는 `기존의 값을 유지한다.`


In [ ]:
# region_2depth_name 컬럼의 값이 '관악구'가 아닌 경우, 해당 컬럼의 다른 값들을 모두 999로 변경
shopsDf['region_2depth_name'].where(shopsDf['region_2depth_name'] == '관악구', 999).to_frame()

In [ ]:
# region_2depth_name 컬럼의 값이 '관악구'가 아닌 경우, 해당 행을 제외한 나머지 행의 모든 컬럼의 값을 999로 변경
shopsDf.where(shopsDf['region_2depth_name'] == '관악구', 999)

## np.where ⭐️
- `np.where(배열에 대한 조건문, 조건이 참일때 값, 조건이 거짓일때 값)` 형태로 사용하며 배열 형태로 반환
- `pd.where` 보다 유연하게 사용 가능


In [ ]:
shopsDf

In [ ]:
# << 어떤 결과가 나오나요? >>
np.where(shopsDf['region_2depth_name'] != '영등포구')

# << 여기에 조건을 충족하는경우와, 아닌경우에 임의의 값을 넣어봅시다. >>
# np.where(shopsDf['region_2depth_name'] != '영등포구', '충족하면_이렇게', '아니면_이렇게')
# shopsDf['region_3depth_name'] = np.where(shopsDf['region_2depth_name'] != '영등포구', '충족하면_이렇게', '아니면_이렇게')
# shopsDf

## df.isin()
- 특정 값에 대한 포함 여부를 확인

In [ ]:
# DataFrame.isin() - shopsDf을 구성하는 각 값들 중에서, '픽업존(메이비카페)', '픽업존(더현대서울)'과 일치하는 경우만 true로 출력
shopsDf.isin(['픽업존(메이비카페)', '픽업존(더현대서울)'])

# << 어떻게 출력될까요? >>
# shopsDf[shopsDf.isin(['픽업존(메이비카페)', '픽업존(더현대서울)'])]

In [ ]:
# Series.isin() - pickup_shop_name 컬럼의 값들 중 '픽업존(메이비카페)', '픽업존(더현대서울)'과 일치하는 경우만 True 출력
shopsDf['pickup_shop_name'].isin(['픽업존(메이비카페)', '픽업존(더현대서울)'])
# shopsDf.isin(['픽업존(메이비카페)', '픽업존(더현대서울)'])

# << 어떻게 출력될까요? >>
shopsDf[shopsDf['pickup_shop_name'].isin(['픽업존(메이비카페)', '픽업존(더현대서울)'])]

## datetime ⭐️
- 공식 문서 참고 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.year.html
- 외부에서 불러온 데이터에서 날짜 형식의 데이터가 있는경우 `datetime 타입으로 변환시켜주시는게 좋습니다.` (데이터 핸들링 편의성 관점)
- datetime 타입의 데이터를 사용할 경우, 사용할 수 있는 기능
  - dt.year: 연도
  - dt.month: 월
  - dt.day: 일
  - dt.week: 주
  - dt.weekofyear: 연중 몇 째주
  - dt.dayofweek: 요일 (월요일: 0, ..., 일요일: 6)
  - dt.weekday: 요일 (dayofweek과 동일)

In [ ]:
ordersDf['order_date'].dt.year
# ordersDf['order_date'].dt.month
# ordersDf['order_date'].dt.day
# ordersDf['order_date'].dt.dayofweek

# 데이터 병합

## pd.concat() ⭐️
- 지정한 DataFrame을 연결하는데 사용

In [ ]:
# << 실습을 위한 shopsDf 데이터 분할 >>
shopsDf_02 = shopsDf.loc[0:2]
shopsDf_34 = shopsDf.loc[3:4]
shopsDf_57 = shopsDf.loc[5:7]

# << concat을 통한 위, 아래 병합 >>
# 각 컬럼 기준으로 같은 이름들끼리 알아서 병합해주고 (컬럼 순서도 상관 X), 병합시 다른 한쪽에 없는 컬럼이 있으면 없는데로 null값을 할당해서 합쳐 줍니다.
pd.concat([shopsDf_02, shopsDf_57, shopsDf_34], ignore_index=True) # ignore_index=True 설정이 있어야, 데이터 병합 후 index가 재설정 됩니다.

In [ ]:
# << 실습을 위한 shopsDf 데이터 분할 >>
shopsDf_left = shopsDf.loc[:, ['pickup_shop_name', 'region_1depth_name', 'region_2depth_name', 'region_3depth_name']]
shopsDf_right = shopsDf.loc[4:, ['h_code', 'x', 'y']]

# << concat을 통한 왼쪽, 오른쪽 병합 >>
# 위아래 병합과 달리 같은 인덱스 끼리 합해줍니다.
pd.concat([shopsDf_left, shopsDf_right], axis=1) # axis = 1 설정을 통해 열방향 병합을 설정!

In [ ]:
shopsDf

## pd.merge() ⭐️
- 공통된 key(컬럼)을 기준으로 DataFraem을 병합하는데 사용
- default 조인 방법은 inner이며, `총 네가지 병합 방식을 제공(left, right, outer, inner)`
- SQL의 JOIN과 유사한 기능

In [ ]:
# << 실습 데이터 확인 >>
salesByShopDf_harf = salesByShopDf.loc[0:4]
salesByShopDf_harf

In [ ]:
# << 이름이 같은 두 컬럼을 조인키로 지정하는 방법 - on >>
pd.merge(salesByShopDf_harf, salesByShopDf, on = 'pickup_shop_name', how = 'left') # left, right, outer, inner

,pickup_shop_name,sales_x,pu_x,sales_y,pu_y
0,픽업존(유승상가),267844000,1929,267844000,1929
1,픽업존(더현대서울),1199500,23,1199500,23
2,픽업존(메이비카페),76700200,653,76700200,653
3,픽업존(GS25 목동3동점),33700500,394,33700500,394
4,픽업존(GS영등포당산점),36577700,547,36577700,547


In [ ]:
# << 이름이 다른 두 컬럼을 조인키로 지정하는 방법 - left_on, right_on >>

# 임의 데이터 분할
a = shopsDf.loc[:, ['pickup_shop_name', 'region_1depth_name', 'region_2depth_name']]
b = shopsDf.loc[:4, ['pickup_shop_name', 'h_code']].rename(columns={'pickup_shop_name':'shop_name'})

# 이름이 다른 컬럼으로, 두 데이터 프레임 join
pd.merge(a, b, left_on = 'pickup_shop_name', right_on = 'shop_name', how = 'left')

# 기타 고급 기능

## df.groupby() ⭐️

In [ ]:
# order_year_month별 합 - 숫자형 컬럼들에대해 모두 sum값이 계산되어 나오지만, 이 데이터에는 숫자형이 컬럼이 payment_amount 하나밖에 없습니다.
ordersDf.groupby('order_year_month').sum()

In [ ]:
# order_year_month별 pickup_shop_name 별 합 - 만약 여러개의 숫자형 컬럼이 있었다면 그룹핑 결과 중 특정 컬럼(e.g. payment_amount)에 대해서만 선택해서 볼 수 있습니다.
ordersDf.groupby(['order_year_month', 'pickup_shop_name'])['payment_amount'].sum()

In [ ]:
# to_frame() 함수를 사용해서, 데이터 프레임 형태로 볼 수 있습니다.
# ㄴ to_frame()은 Seires를 DataFrame으로 변환하는 함수이며
# ㄴ 아래 코드 전체를 pd.DataFrame()으로 감싸는 방법도 있습니다.
ordersDf.groupby(['order_year_month', 'pickup_shop_name'])['payment_amount'].sum().to_frame()

In [ ]:
# reset_index()를 사용해서 index를 초기화하고 새로운 데이터 프레임을 만들 수 있습니다.
ordersDf.groupby(['order_year_month', 'pickup_shop_name'])['payment_amount'].sum().reset_index()

In [ ]:
# << 다중 통계량 계산 >>

multiStatsDf = ordersDf.groupby('order_year_month').agg(['sum', 'mean', 'median', 'max', 'min', 'count', 'nunique', 'std', 'var']).reset_index()
multiStatsDf

In [ ]:
# << (참고) 멀티 컬럼에 접근하는 방법 - 어떤 차이가 있는지 한줄식 실행해 보세요 ~ ☺️ >>

multiStatsDf['payment_amount']['sum']
multiStatsDf[('payment_amount', 'sum')] # 튜플형태로 접근
multiStatsDf['payment_amount'][['sum', 'mean']]
multiStatsDf.loc[[1,6], 'payment_amount'][['sum', 'max']]

## df.pivot_table()
- `index, colums, values`를 지정하여 테이블을 피벗합니다.
  - index : 왼쪽의 기준 행으로 두고 싶은 컬럼
  - columns : 상단에 나열하고 싶은 컬럼
  - values : 안에 표기하고 싶은 데이터
- `aggfunc`는 defaul로 sum이 할당되어 있으며, 변경 및 추가 가능


In [ ]:
shopsDf.pivot_table(index = 'pickup_shop_name' , columns= 'region_2depth_name', values = 'x')

In [ ]:
shopsDf.pivot_table(index = 'region_2depth_name' , columns= 'region_1depth_name', values = 'x', aggfunc=['sum', 'min'])

## df.melt()
- `id_vars, value_vars`를 지정하여 테이블 형태를 변경합니다.
  - id_vers : 왼쪽의 기준 행으로 두고 싶은 변수 
  - value_vars : 녹이고자 하는 컬럼
  - var_name : 변경하고자 하는 variable 컬럼의 이름
  - value_name : 변경하고자 하는 value 컬럼의 이름


In [ ]:
shopsDf

In [ ]:
# 우선 어떻게 녹아내리는지 확인해 볼까요?
shopsDf.melt(id_vars='pickup_shop_name')

In [ ]:
# pickup_shop_name을 기준으로 region_1depth_name 컬럼 하나만 녹이고 싶다면?
shopsDf.melt(id_vars='pickup_shop_name', value_vars='region_1depth_name')

In [ ]:
# pickup_shop_name을 기준으로 h_code, x, y 컬럼을 녹이고 싶다면?
shopsDf.melt(id_vars='pickup_shop_name', value_vars=['h_code', 'x', 'y'])# var_name = 'category', value_name = 'index'

# pandasql 
- 해당 라이브러리를 사용하면, python에서 새롭게 생성된 DataFrame도 SQL 문법에 맞춰 핸들링 할 수 있기 때문에, pandas 문법이 갑자기 생각이 나지 않더라도 유연하게 대처할 수 있습니다.
- 단 bigquery에서 사용되는 일부 함수의 경우 해당 기능을 제공하지 않을 수 있습니다. 
- https://pypi.org/project/pandasql/
- https://github.com/yhat/pandasql/
- SQLite - SQL Function : https://www.sqlite.org/lang_corefunc.html
- pandasql uses SQLite syntax. Any pandas dataframes will be automatically detected by pandasql. You can query them as you would any regular SQL table.

In [ ]:
!pip install pandasql

In [ ]:
!pip show pandasql

Name: pandasql
Version: 0.7.3
Summary: sqldf for pandas
Home-page: https://github.com/yhat/pandasql/
Author: Greg Lamp
Author-email: greg@yhathq.com
License: Copyright (c) 2013 Yhat, Inc.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.

In [ ]:
from pandasql import sqldf
dfsql = lambda q: sqldf(q, globals())

basicIndexDf = dfsql("""
with login_users as (
  select login_year_month, count(distinct user_id) as mau
  from loginUsersDf
  group by login_year_month
),

orders as (
  select order_year_month, sum(payment_amount) as sales, count(distinct user_id) as pu
  from ordersDf
  group by order_year_month
)

select a.login_year_month, a.mau, b.sales, b.pu, round(b.sales/b.pu, 2) as arppu, round(round(b.sales)/b.pu, 2) as arppu_re
from login_users as a
left join orders as b
on a.login_year_month = b.order_year_month
order by a.login_year_month
""")

basicIndexDf

In [ ]:
basicIndexDf.info()